In [ ]:
!unzip /content/drive/MyDrive/archive.zip

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 데이터 경로 설정
train_data_dir = '/content/chest_xray/train'
val_data_dir = '/content/chest_xray/val'
test_data_dir = '/content/chest_xray/test'

# 데이터 생성기 설정
img_size = 224
batch_size = 32

# 그레이 스케일, 해상도 축소
train_datagen = ImageDataGenerator(rescale=1./255) # 훈련 데이터 전처리
val_datagen = ImageDataGenerator(rescale=1./255) # 검증 데이터 전처리
test_datagen = ImageDataGenerator(rescale=1./255) # 테스트 데이터 전처리

# 훈련 데이터 설정, 라벨링
train_data = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training')

# 검증 데이터 설정, 라벨링
val_data = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary') 

# 테스트 데이터 설정, 라벨링
test_data = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary')

# 전이학습 모델 생성
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3)) # VGG16 모델 로드

x = base_model.output
x = GlobalAveragePooling2D()(x) # 특징 추출
x = Dense(256, activation='relu')(x) # 전결합층 추가
predictions = Dense(1, activation='sigmoid')(x) # 출력층 추가

model = Model(inputs=base_model.input, outputs=predictions) # 전이학습 모델 생성


# 사전 훈련된 모델 동결
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
epochs = 10
steps_per_epoch = 500 // batch_size  # 2개(노말,폐렴) 클래스당 500개씩 사용
validation_steps = val_data.samples // batch_size

model.fit(
    train_data, # 훈련 데이터
    steps_per_epoch=steps_per_epoch, # 에포크 당 스텝 수 10
    epochs=epochs, # 전체 에포크 수 10
    validation_data=val_data, # 검증 데이터
    validation_steps=validation_steps) # 검증 단계 수

# 미세조정
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    train_data, 
    steps_per_epoch=steps_per_epoch, 
    epochs=epochs,
    validation_data=val_data, 
    validation_steps=validation_steps) 

# 모델 평가 
test_loss, test_accuracy = model.evaluate(test_data) 
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10
15/15 [==============================] - 8s 452ms/step - loss: 0.5271 - accuracy: 0.7312
Epoch 2/10
15/15 [==============================] - 6s 385ms/step - loss: 0.4200 - accuracy: 0.8125
Epoch 3/10
15/15 [==============================] - 7s 465ms/step - loss: 0.3370 - accuracy: 0.8479
Epoch 4/10
15/15 [==============================] - 6s 421ms/step - loss: 0.2916 - accuracy: 0.8896
Epoch 5/10
15/15 [==============================] - 6s 364ms/step - loss: 0.2214 - accuracy: 0.9083
Epoch 6/10
15/15 [==============================] - 6s 406ms/step - loss: 0.2394 - accuracy: 0.8958
Epoch 7/10
15/15 [==============================] - 6s 428ms/step - loss: 0.2375 - accuracy: 0.8938
Epoch 8/10
15/15 [==============================] - 6s 369ms/step - loss: 0.2134 - accuracy: 0.9229
Epoch 9/10
15/15 [==============================] - 6s 359ms/step - loss: 0.1

Epoch 1/10
15/15 [==============================] - 10s 478ms/step - loss: 2.0459 - accuracy: 0.7479
Epoch 2/10
15/15 [==============================] - 6s 398ms/step - loss: 0.5461 - accuracy: 0.7208
Epoch 3/10
15/15 [==============================] - 7s 479ms/step - loss: 0.5739 - accuracy: 0.7396
Epoch 4/10
15/15 [==============================] - 6s 405ms/step - loss: 0.3308 - accuracy: 0.7437
Epoch 5/10
15/15 [==============================] - 8s 522ms/step - loss: 0.3092 - accuracy: 0.8188
Epoch 6/10
15/15 [==============================] - 6s 406ms/step - loss: 0.2702 - accuracy: 0.9021
Epoch 7/10
15/15 [==============================] - 8s 500ms/step - loss: 0.2917 - accuracy: 0.8729
Epoch 8/10
15/15 [==============================] - 6s 425ms/step - loss: 0.3015 - accuracy: 0.8917
Epoch 9/10
15/15 [==============================] - 7s 420ms/step - loss: 0.2718 - accuracy: 0.8979
Epoch 10/10
20/20 [==============================] - 7s 311ms/step - loss: 1.1039 - accuracy: 0.713